# Choosing the best city for my new office

### Import libraries

In [1]:
from pymongo import GEOSPHERE
from pymongo import MongoClient
import pandas as pd
import sys
sys.path.append('../')
import src.cleaning as cl

In [2]:
client = MongoClient('localhost:27017')
db = client.get_database('ironhack')

### Getting the distances from bars, airports and schools from each city main coordinate.

In [3]:
madrid = db.get_collection('Madrid')
vancouver = db.get_collection('Vancouver')
trentino = db.get_collection('Trentino-Alto Adige')

In [4]:
mad_coo = {'type': 'Point', 'coordinates': [-3.62207, 40.39818]}
van_coo = {'type': 'Point', 'coordinates': [-123.10299, 49.24847]}
tre_coo = {'type': 'Point', 'coordinates': [11.33492, 46.47754]}

##### MADRID 

In [6]:
query_mad = [{"$geoNear": {
            "near": mad_coo['coordinates'],
            "distanceField": "distance",
            "maxDistance": 30000,
            "distanceMultiplier" :6371,
            "spherical": True}}]
dist_mad = list(madrid.aggregate(query_mad))
dist_mad = pd.DataFrame(dist_mad)
dist_mad = dist_mad.drop(columns=['_id', 'geometry'])
dist_mad.insert(0, 'city', 'Madrid')
dist_mad.head(1)

,city,types,name,longitude,lattitude,distance
0,Madrid,bar,Bar,-3.624506,40.397586,0.21657


#### VANCOUVER 

In [8]:
query_van = [{"$geoNear": {
            "near": van_coo['coordinates'],
            "distanceField": "distance",
            "maxDistance": 30000,
            "distanceMultiplier" :6371,
            "spherical": True}}]
dist_van = list(trentino.aggregate(query_van))
dist_van = pd.DataFrame(dist_van)
dist_van = dist_van.drop(columns=['_id', 'geometry'])
dist_van.insert(0, 'city', 'Vancouver')
dist_van.head(1)

,city,types,name,longitude,lattitude,distance
0,Vancouver,airport,Aliporto Elisoccorso H Merano,11.149452,46.678572,8472.631662


#### TRENTINO-ALTO ADIGE

In [10]:
query_tre = [{"$geoNear": {
            "near": tre_coo['coordinates'],
            "distanceField": "distance",
            "maxDistance": 30000,
            "distanceMultiplier" :6371,
            "spherical": True}}]
dist_tre = list(trentino.aggregate(query_tre))
dist_tre = pd.DataFrame(dist_tre)
dist_tre = dist_tre.drop(columns=['_id', 'geometry'])
dist_tre.insert(0, 'city', 'Trentino-Alto Adige')
dist_tre.head(1)

,city,types,name,longitude,lattitude,distance
0,Trentino-Alto Adige,bar,Top Bar,11.333633,46.476981,0.116477


# Scores

### The city where the sum of the aggregated distances of the nearest: airport, bar and school from the main point, will win.

In [12]:
df_cities = pd.concat([dist_mad, dist_van, dist_tre]).reset_index(drop=True)
df_cities['types'] = df_cities['types'].str.replace('secondary_school','school' )

In [13]:
df_cities = df_cities.groupby(['city', 'types']).agg({'distance': 'min'}).unstack()
df_cities['total'] = df_cities.sum(numeric_only=True, axis=1)
df_cities

distance                                   total
types                    airport          bar       school              
city                                                                    
Madrid                  4.983031     0.216570     5.930955     11.130556
Trentino-Alto Adige     1.717930     0.116477     0.687058      2.521465
Vancouver            8472.631662  8497.804572  8493.286437  25463.722670

### Trentinto-Alto Adige in Italy, is the most convenient city for the new office.